# Electricity demand in Quebec

We now focus on electricity consumption in Quebec.



## Hydro-Québec open data

Since 2019, Hydro-Québec shares data on various aspects regarding its operations, at the page https://www.hydroquebec.com/documents-data/open-data/.

Here, we are interested in the historical demand: https://www.hydroquebec.com/documents-data/open-data/history-electricity-demand-quebec/. In particular, we can track the hourly total demand in Quebec, noting that
> Electricity demand for one hour corresponds to the total average demand during that hour.

> The data is determined at the end of a time period. For example, the average hourly demand associated with 2019‑01‑01 2:00 is the average of the data collected from 2019‑01‑01 1:05 to 2019‑01‑01 2:00.

We will analyze the data using Pandas and Plotly.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

## A first forecasting model

The model we will built is highly simplified as the total Quebec consumption is taken into account, along with a single weather station.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
import statsmodels.api as sm
from data.get_data import HQ_data
from statsmodels.tsa.arima.model import ARIMA

class Fourier_series:

    def get_predictions(self, data, train_start, train_end, test):

        pf = PolynomialFeatures(degree=2)

        data = data.loc[train_start:test, :].copy()

        # get fourier series features
        ff_week = self.get_fourier_features(5, 7, data.loc[:, "day"])
        ff_24h = self.get_fourier_features(5, 24, data.loc[:, "hour"])
        fourier_features = pd.concat([ff_week, ff_24h], ignore_index=True, axis=1)

        # add extra predictors
        features = ["scaled_temp", "temp_lag_15", "temp_index_15", 
                    "demand_lag_24", "is_clear", "rel_hum", "scaled_temp_diff_24", "scaled_temp_diff_48"]

        X = fourier_features
        X[features] = data.loc[:, features]
        X.columns = X.columns.astype(str)
        
        # train model

        X_train = pf.fit_transform(X.loc[train_start:train_end, :])
        y_train = data.loc[train_start:train_end, "log_demand"]

        model = sm.OLS(y_train, X_train).fit()

        # get forecast
        X_test = pf.fit_transform(np.array(X.loc[test, :]).reshape(1, -1))
        forecast  = np.exp(model.predict(X_test))

        return forecast
    

    def get_fourier_features(self, n_order, period, values):
        fourier_features = pd.DataFrame(
            {
                f"fourier_{func}_order_{order}_{period}": getattr(np, func)(
                    2 * order * np.pi * values / period
                )
                for order in range(1, n_order + 1)
                for func in ("sin", "cos")
            }
        )
        return fourier_features

In [ ]:
from data.get_data import HQ_data
from models.regression_splines import SplineRegression
# from models.fourier_series import Fourier_series
from models.arma import ARMA_model
from models.mlp import MLP_model

class Simulation:

    def __init__(self, num_iters, train_start, train_end):

        self.num_iters = num_iters
        self.train_start = train_start
        self.train_end = train_end
        self.test = self.train_end + datetime.timedelta(days=1)
        self.data = HQ_data()
        self.data = self.data.get_history()

    def get_prediction(self, train_start, train_end, test):

        """ 
        Implement algorithm or call model here. 
        Model should have get_predictions method that takes in data, train_start, train_end, test 
        and returns a single forecast for the time step test: 24 hour after train_end

        """

        """mlp = MLP_model()
        forecasts = mlp.get_predictions(self.data, train_start, train_end, test)"""

        """spline_reg = SplineRegression()
        forecasts = spline_reg.get_predictions(self.data, train_start, train_end, test)"""

        fourier = Fourier_series()
        forecasts = fourier.get_predictions(self.data, train_start, train_end, test)

        """arma = ARMA_model()
        forecasts = arma.get_predictions(self.data, train_start, train_end, test)"""

        return forecasts

    def run_simulation(self):
        
        train_start = self.train_start
        train_end = self.train_end
        test = self.test

        forecasts = []

        for i in range(self.num_iters):

            forecast = self.get_prediction(train_start, train_end, test)
            forecasts.append(forecast)

            print("********************************************")
            print("At iteration "+ str(i))
            print("forecast: ", forecast)
            print("********************************************")

            # We increment the various times by one hour.
            train_start = train_start + datetime.timedelta(hours=1)
            train_end = train_end + datetime.timedelta(hours=1)
            test = test + datetime.timedelta(hours=1)

        return np.array(forecasts).flatten()

    def plot_sim_results(self, forecasts):

        sim_start = self.train_end + datetime.timedelta(days=1)
        sim_end = sim_start + datetime.timedelta(hours = self.num_iters) - datetime.timedelta(hours=1)

        results = self.data.loc[sim_start:sim_end, ["demand", "scaled_temp"]]
        results["forecast"] = forecasts

        # save forecasts to csv
        results.to_csv("results\\results_fourier_2021.csv")

        residuals = results.loc[:, "demand"] - results.loc[:, "forecast"]

        print("RMSE")
        print(np.sqrt(np.mean(residuals**2)))

        print("MAPE")
        print(np.mean(abs(residuals)/results.loc[:, "demand"]))

        print("Percentage within 1000 mwh")
        print(sum(list(map(lambda x: int(x <= 1000), abs(residuals))))/len(residuals))

        print("Percentage within 500 mwh")
        print(sum(list(map(lambda x: int(x <= 500), abs(residuals))))/len(residuals))

        plt.plot(results.loc[:, "demand"], label="Demand")
        plt.plot(results.loc[:, "forecast"], label="Forecast")
        plt.legend()
        plt.title("24 hour ahead energy demand forecast for year 2022")
        plt.show()

In [ ]:
train_start = datetime.datetime(2019, 1, 1, 0, 0, 0)
train_end = datetime.datetime(2020, 12, 31, 23, 0, 0)

# We train the model at each hour increment, for 365 days.
sim = Simulation(365 * 24, train_start, train_end)
forecasts = sim.run_simulation()
sim.plot_sim_results(forecasts)

In [ ]:
sim.plot_sim_results(forecasts)